In [20]:
import pandas as pd
import matplotlib.pyplot as plt

from pandas import read_csv

df = read_csv("../data/data_collection_202412_202412.csv", low_memory=False, encoding='latin1', delimiter=';')

In [21]:

# Step 1: Remove low variance columns (protect 'Target')
def remove_low_variance_columns(df, variance_threshold=0.01, protected_cols=['Target']):
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
    numeric_cols = [col for col in numeric_cols if col not in protected_cols]

    if len(numeric_cols) > 0:
        variances = df[numeric_cols].var()
        low_variance_cols = variances[variances < variance_threshold].index.tolist()
    else:
        low_variance_cols = []

    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    cat_cols = [col for col in cat_cols if col not in protected_cols]
    dominant_cat_cols = []

    for col in cat_cols:
        if len(df[col].value_counts()) > 0:
            dominant_pct = df[col].value_counts(normalize=True).iloc[0]
            if dominant_pct > 0.95:
                dominant_cat_cols.append(col)

    return low_variance_cols, dominant_cat_cols


# Run cleaning
low_var_numeric, dominant_cat = remove_low_variance_columns(df, protected_cols=['Target'])
columns_to_drop_step1 = low_var_numeric + dominant_cat
df_step1 = df.drop(columns=columns_to_drop_step1)

# Step 2: Remove high missing (protect 'Target')
missing_summary = pd.DataFrame({
    'missing_count': df_step1.isnull().sum(),
    'missing_percentage': (df_step1.isnull().sum() / len(df_step1)) * 100
})

high_missing_cols = missing_summary[
    (missing_summary['missing_percentage'] > 97) &
    (missing_summary.index != 'Target')
    ].index.tolist()

df_cleaned = df_step1.drop(columns=high_missing_cols)

print(f"FINAL RESULT:")
print(f"Original columns: {df.shape[1]}")
print(f"After cleaning: {df_cleaned.shape[1]}")
print(f"Removed: {df.shape[1] - df_cleaned.shape[1]} columns")
print(f"Target preserved: {'Target' in df_cleaned.columns}")

FINAL RESULT:
Original columns: 918
After cleaning: 557
Removed: 361 columns
Target preserved: True


In [22]:
# Identify and fix other numeric columns stored as strings
print("Fixing all numeric columns stored as strings...")

# Find columns that should be numeric but are stored as objects
numeric_pattern_cols = [col for col in df_cleaned.columns if
                        any(prefix in col for prefix in ['QTD_', 'VAL_', 'TAX_', 'PCT_', 'IND_'])]

for col in numeric_pattern_cols:
    if df_cleaned[col].dtype == 'object':
        print(f"Converting {col} to numeric...")
        # Handle comma decimal separators
        df_cleaned[col] = df_cleaned[col].astype(str).str.replace(',', '.')
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

print("Data type conversion completed!")

Fixing all numeric columns stored as strings...
Converting IND_T_S_GLOBAL to numeric...
Converting IND_PSIN_PCOL_ENI to numeric...
Converting QTD_ANOS_ANTIG_CLIENTE to numeric...
Converting VAL_MARG_TECNICA_MED_PREF to numeric...
Converting TAX_SINISTR_MED_PREF to numeric...
Converting TAX_CAPTACAO_MED_PREF to numeric...
Converting TAX_ANULACAO_MED_PREF to numeric...
Converting VAL_IND_COMPET_MED_PREF to numeric...
Converting PCT_FAM_MAIS_APL_MED_PREF to numeric...
Converting PCT_FAM_MA_PCA_MED_PREF to numeric...
Converting VAL_MAX_MATUR_VIG to numeric...
Converting VAL_MAX_MATUR_VIG_AP to numeric...
Converting VAL_MAX_MATUR_VIG_ATCP to numeric...
Converting VAL_MAX_MATUR_VIG_AUTO to numeric...
Converting VAL_MAX_MATUR_VIG_CAP to numeric...
Converting VAL_MAX_MATUR_VIG_MRH to numeric...
Converting VAL_MAX_MATUR_VIG_PPR to numeric...
Converting VAL_MAX_MATUR_VIG_SD to numeric...
Converting VAL_MAX_MATUR_VIG_VDR to numeric...
Converting VAL_PCA_VIG_ENTRE_12_24M_SFIN to numeric...
Convert

In [23]:


# Step 1: Remove low variance columns (protect 'Target')
def remove_low_variance_columns(df, variance_threshold=0.01, protected_cols=['Target']):
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns
    numeric_cols = [col for col in numeric_cols if col not in protected_cols]

    if len(numeric_cols) > 0:
        variances = df[numeric_cols].var()
        low_variance_cols = variances[variances < variance_threshold].index.tolist()
    else:
        low_variance_cols = []

    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    cat_cols = [col for col in cat_cols if col not in protected_cols]
    dominant_cat_cols = []

    for col in cat_cols:
        if len(df[col].value_counts()) > 0:
            dominant_pct = df[col].value_counts(normalize=True).iloc[0]
            if dominant_pct > 0.95:
                dominant_cat_cols.append(col)

    return low_variance_cols, dominant_cat_cols


# Run cleaning
low_var_numeric, dominant_cat = remove_low_variance_columns(df, protected_cols=['Target'])
columns_to_drop_step1 = low_var_numeric + dominant_cat
df_step1 = df.drop(columns=columns_to_drop_step1)

# Step 2: Remove high missing (protect 'Target')
missing_summary = pd.DataFrame({
    'missing_count': df_step1.isnull().sum(),
    'missing_percentage': (df_step1.isnull().sum() / len(df_step1)) * 100
})

high_missing_cols = missing_summary[
    (missing_summary['missing_percentage'] > 97) &
    (missing_summary.index != 'Target')
    ].index.tolist()

df_cleaned = df_step1.drop(columns=high_missing_cols)

print(f"FINAL RESULT:")
print(f"Original columns: {df.shape[1]}")
print(f"After cleaning: {df_cleaned.shape[1]}")
print(f"Removed: {df.shape[1] - df_cleaned.shape[1]} columns")
print(f"Target preserved: {'Target' in df_cleaned.columns}")

FINAL RESULT:
Original columns: 918
After cleaning: 557
Removed: 361 columns
Target preserved: True


In [24]:
print(f"TARGET in original df: {'TARGET' in df.columns}")
print(f"TARGET in cleaned df: {'TARGET' in df_cleaned.columns}")

# If TARGET was accidentally removed, let's check why
if 'Target' in df.columns and 'Target' not in df_cleaned.columns:
    print("TARGET was accidentally removed during cleaning!")
    print("Let's check TARGET's characteristics:")
    print(f"TARGET missing %: {(df['TARGET'].isnull().sum() / len(df)) * 100:.2f}%")
    if df['Target'].dtype in ['int64', 'float64']:
        print(f"TARGET variance: {df['TARGET'].var()}")

TARGET in original df: False
TARGET in cleaned df: False


In [25]:
# Step 3: Target Analysis
print("\nSTEP 3 - Target Variable Analysis:")
print("=" * 40)
print("Target distribution:")
print(df_cleaned['Target'].value_counts())
print("\nTarget percentages:")
target_dist = df_cleaned['Target'].value_counts(normalize=True)
print(target_dist)
print(f"\nMySavings cross-sell rate: {target_dist[1]:.2%}")


STEP 3 - Target Variable Analysis:
Target distribution:
Target
0    377206
1       716
Name: count, dtype: int64

Target percentages:
Target
0    0.998105
1    0.001895
Name: proportion, dtype: float64

MySavings cross-sell rate: 0.19%


In [26]:
print("STEP 4 - Duplicate Check:")
print("=" * 40)

# Check for duplicate rows
duplicate_rows = df_cleaned.duplicated().sum()
print(f"Duplicate rows: {duplicate_rows}")

# Check for duplicate customer IDs
duplicate_customers = df_cleaned['ID_CLIENTE_ANON'].duplicated().sum()
print(f"Duplicate customer IDs: {duplicate_customers}")

# If duplicates exist, remove them
if duplicate_rows > 0:
    print("Removing duplicate rows...")
    df_cleaned = df_cleaned.drop_duplicates()
    print(f"Rows after duplicate removal: {df_cleaned.shape[0]}")

if duplicate_customers > 0:
    print("Removing duplicate customers (keeping first occurrence)...")
    df_cleaned = df_cleaned.drop_duplicates(subset=['ID_CLIENTE_ANON'], keep='first')
    print(f"Rows after customer dedup: {df_cleaned.shape[0]}")

STEP 4 - Duplicate Check:
Duplicate rows: 0
Duplicate customer IDs: 34
Removing duplicate customers (keeping first occurrence)...
Rows after customer dedup: 377888


In [31]:
print("STEP 5 - Data Type Fixes and Quality Check:")
print("=" * 40)

# Check data types of key columns
print("Key column data types:")
key_cols = ['QTD_IDADE', 'QTD_ANOS_ANTIG_CLIENTE', 'Target']
for col in key_cols:
    if col in df_cleaned.columns:
        print(f"  {col}: {df_cleaned[col].dtype}")

# Fix numeric columns that are stored as strings
print(f"\nFixing data types...")

# Age should be numeric
if df_cleaned['QTD_IDADE'].dtype == 'object':
    print("Converting QTD_IDADE to numeric...")
    df_cleaned['QTD_IDADE'] = pd.to_numeric(df_cleaned['QTD_IDADE'], errors='coerce')

# Customer tenure should be numeric
if df_cleaned['QTD_ANOS_ANTIG_CLIENTE'].dtype == 'object':
    print("Converting QTD_ANOS_ANTIG_CLIENTE to numeric...")
    # Handle potential comma as decimal separator
    df_cleaned['QTD_ANOS_ANTIG_CLIENTE'] = df_cleaned['QTD_ANOS_ANTIG_CLIENTE'].astype(str).str.replace(',', '.')
    df_cleaned['QTD_ANOS_ANTIG_CLIENTE'] = pd.to_numeric(df_cleaned['QTD_ANOS_ANTIG_CLIENTE'], errors='coerce')

# Now check the data quality
print(f"\nAge distribution:")
print(df_cleaned['QTD_IDADE'].describe())
age_issues = (df_cleaned['QTD_IDADE'] < 18) | (df_cleaned['QTD_IDADE'] > 80)
print(f"Ages outside 18-80 range: {age_issues.sum()}")

print(f"\nCustomer tenure distribution:")
print(df_cleaned['QTD_ANOS_ANTIG_CLIENTE'].describe())
tenure_issues = df_cleaned['QTD_ANOS_ANTIG_CLIENTE'] < 0
print(f"Negative tenure values: {tenure_issues.sum()}")

# Check for NaN values created during conversion
print(f"\nNaN values created during conversion:")
print(f"  QTD_IDADE: {df_cleaned['QTD_IDADE'].isnull().sum()}")
print(f"  QTD_ANOS_ANTIG_CLIENTE: {df_cleaned['QTD_ANOS_ANTIG_CLIENTE'].isnull().count()}")

STEP 5 - Data Type Fixes and Quality Check:
Key column data types:
  QTD_IDADE: int64
  QTD_ANOS_ANTIG_CLIENTE: float64
  Target: int64

Fixing data types...

Age distribution:
count    377888.000000
mean         48.170458
std          13.082849
min          19.000000
25%          38.000000
50%          48.000000
75%          57.000000
max          79.000000
Name: QTD_IDADE, dtype: float64
Ages outside 18-80 range: 0

Customer tenure distribution:
count    372054.000000
mean         16.516462
std          10.968809
min          -0.080000
25%           6.830000
50%          15.750000
75%          24.330000
max         112.080000
Name: QTD_ANOS_ANTIG_CLIENTE, dtype: float64
Negative tenure values: 3

NaN values created during conversion:
  QTD_IDADE: 0
  QTD_ANOS_ANTIG_CLIENTE: 377888


In [32]:
print("STEP 6 - Final Dataset Summary:")
print("=" * 40)
print(f"Dataset shape: {df_cleaned.shape}")
print(f"Features available: {df_cleaned.shape[1] - 1}")
print(f"MySavings cross-sell rate: {df_cleaned['Target'].mean():.2%}")

# Data types after cleaning
print(f"\nData types distribution:")
print(df_cleaned.dtypes.value_counts())

# Quick missing values check
missing_summary = (df_cleaned.isnull().sum() / len(df_cleaned) * 100).sort_values(ascending=False)
cols_with_missing = missing_summary[missing_summary > 0]
print(f"\nColumns with missing values: {len(cols_with_missing)}")
if len(cols_with_missing) > 0:
    print("Top 5 columns with missing values:")
    print(cols_with_missing.head())

STEP 6 - Final Dataset Summary:
Dataset shape: (377888, 557)
Features available: 556
MySavings cross-sell rate: 0.19%

Data types distribution:
int64      196
float64    189
object     172
Name: count, dtype: int64

Columns with missing values: 336
Top 5 columns with missing values:
VAL_MAX_MATUR_VIG_ATCP    96.846155
VAL_PCA_APL_VIG_ATCP      96.846155
VAL_MIN_MATUR_VIG_ATCP    96.846155
QTD_MESES_CHURN_AUTO      96.592641
VAL_MIN_MATUR_ANU_CAP     96.240685
dtype: float64


In [33]:
print("STEP 7 - Target Analysis:")
print("=" * 40)

# Target distribution
target_counts = df_cleaned['Target'].value_counts()
target_pct = df_cleaned['Target'].value_counts(normalize=True)

print("Target distribution:")
print(f"  No MySavings (0): {target_counts[0]:,} ({target_pct[0]:.1%})")
print(f"  Bought MySavings (1): {target_counts[1]:,} ({target_pct[1]:.1%})")

# Check if dataset is imbalanced (important for modeling)
if target_pct[1] < 0.05:
    print("⚠️  Highly imbalanced dataset - will need special handling in modeling")
elif target_pct[1] < 0.20:
    print("⚠️  Imbalanced dataset - consider this in modeling approach")
else:
    print("✓ Reasonably balanced dataset")

STEP 7 - Target Analysis:
Target distribution:
  No MySavings (0): 377,185 (99.8%)
  Bought MySavings (1): 703 (0.2%)
⚠️  Highly imbalanced dataset - will need special handling in modeling


In [34]:
print("STEP 8 - Demographics vs Target:")
print("=" * 40)

# Age analysis
print("Age analysis:")
age_stats = df_cleaned.groupby('Target')['QTD_IDADE'].agg(['mean', 'median', 'std'])
print(age_stats)

# Customer tenure analysis
print(f"\nCustomer tenure analysis:")
tenure_stats = df_cleaned.groupby('Target')['QTD_ANOS_ANTIG_CLIENTE'].agg(['mean', 'median', 'std'])
print(tenure_stats)

# Gender analysis (if available)
if 'DSC_SEXO' in df_cleaned.columns:
    print(f"\nGender vs Target:")
    gender_crosstab = pd.crosstab(df_cleaned['DSC_SEXO'], df_cleaned['Target'], normalize='index')
    print(gender_crosstab)

STEP 8 - Demographics vs Target:
Age analysis:
             mean  median        std
Target                              
0       48.174148    48.0  13.082867
1       46.190612    45.0  12.931138

Customer tenure analysis:
             mean  median        std
Target                              
0       16.516612   15.75  10.969189
1       16.436408   15.71  10.771283

Gender vs Target:
Target              0         1
DSC_SEXO                       
F-FEMININO   0.998112  0.001888
M-MASCULINO  0.998161  0.001839


In [35]:
print("STEP 9 - Feature Engineering for Imbalanced Cross-sell:")
print("=" * 50)

# First, let's identify feature categories we have
feature_categories = {}

# Demographic features
demo_cols = [col for col in df_cleaned.columns if
             any(x in col.upper() for x in ['IDADE', 'SEXO', 'CIVIL', 'PROF', 'HABILITACOES'])]
feature_categories['Demographics'] = demo_cols

# Geographic features
geo_cols = [col for col in df_cleaned.columns if
            any(x in col.upper() for x in ['DISTRITO', 'CONCELHO', 'NUT', 'FREGUESIA'])]
feature_categories['Geographic'] = geo_cols

# Digital behavior
digital_cols = [col for col in df_cleaned.columns if
                any(x in col.upper() for x in ['LOGIN', 'APP', 'DIGITAL', 'INTERACT'])]
feature_categories['Digital'] = digital_cols

# Insurance products/policies
insurance_cols = [col for col in df_cleaned.columns if
                  any(x in col.upper() for x in ['APL', 'VIG', 'SEGUROS', 'APOLICE'])]
feature_categories['Insurance'] = insurance_cols

# Financial/Value features
financial_cols = [col for col in df_cleaned.columns if
                  any(x in col.upper() for x in ['VAL', 'CLV', 'SCORE', 'MARGEM', 'PCA'])]
feature_categories['Financial'] = financial_cols

print("Available feature categories:")
for category, cols in feature_categories.items():
    available = [col for col in cols if col in df_cleaned.columns and col != 'Target']
    print(f"  {category}: {len(available)} features")
    if len(available) > 0:
        print(f"    Examples: {available[:3]}")

STEP 9 - Feature Engineering for Imbalanced Cross-sell:
Available feature categories:
  Demographics: 7 features
    Examples: ['DSC_ESTADO_CIVIL', 'DSC_SEXO', 'DSC_SITUACAO_PROF']
  Geographic: 8 features
    Examples: ['DSC_CONCELHO', 'DSC_DISTRITO', 'COD_NUT2']
  Digital: 62 features
    Examples: ['IND_LOGIN_APP_MYFID', 'IND_LOGIN_APP_DRIVE', 'IND_LOGIN_APP_MYMLT']
  Insurance: 176 features
    Examples: ['DSC_FAM_MAIS_APL_MED_PREF', 'PCT_FAM_MAIS_APL_MED_PREF', 'QTD_APL_VIG_T_S']
  Financial: 89 features
    Examples: ['VAL_MARG_TECNICA_MED_PREF', 'VAL_IND_COMPET_MED_PREF', 'DSC_FAM_MA_PCA_MED_PREF']


In [36]:
print("STEP 10 - Age Feature Engineering:")
print("=" * 40)


# Create age groups (common in insurance cross-sell)
def create_age_groups(df):
    df = df.copy()

    # Create age bins
    age_bins = [18, 25, 35, 45, 55, 65, 80]
    age_labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']

    df['Age_Group'] = pd.cut(df['QTD_IDADE'], bins=age_bins, labels=age_labels, right=False)

    return df


df_engineered = create_age_groups(df_cleaned)

# Check age group distribution vs target
print("Age groups vs Target rate:")
age_target_analysis = df_engineered.groupby('Age_Group')['Target'].agg(['count', 'sum', 'mean']).round(4)
age_target_analysis.columns = ['Total_Customers', 'MySavings_Buyers', 'Conversion_Rate']
print(age_target_analysis)

# Find best performing age groups
best_age_groups = age_target_analysis.sort_values('Conversion_Rate', ascending=False)
print(f"\nBest converting age groups:")
print(best_age_groups.head(3))

STEP 10 - Age Feature Engineering:
Age groups vs Target rate:
           Total_Customers  MySavings_Buyers  Conversion_Rate
Age_Group                                                    
18-24                 5701                21           0.0037
25-34                59409               122           0.0021
35-44                87964               189           0.0021
45-54               108110               167           0.0015
55-64                68871               142           0.0021
65+                  47833                62           0.0013

Best converting age groups:
           Total_Customers  MySavings_Buyers  Conversion_Rate
Age_Group                                                    
18-24                 5701                21           0.0037
25-34                59409               122           0.0021
35-44                87964               189           0.0021


In [37]:
print("FEATURE ENGINEERING")
print('=' * 42)

FEATURE ENGINEERING


In [38]:
print("STEP 11 - Insurance Product Features:")
print("=" * 50)

# Find all insurance product columns
insurance_cols = [col for col in df_engineered.columns if 'QTD_APL_VIG' in col and col != 'Target']
print(f"Found {len(insurance_cols)} insurance product columns")

# Show first 5 to understand the data
print(f"\nFirst 5 insurance columns sample:")
for col in insurance_cols[:5]:
    print(f"  {col}:")
    print(f"    Data type: {df_engineered[col].dtype}")
    print(f"    Non-null values: {df_engineered[col].notna().sum()}")
    print(f"    Sample values: {df_engineered[col].dropna().head(3).tolist()}")
    print()

# Check if they're numeric and create total count
numeric_insurance_cols = []
for col in insurance_cols:
    if df_engineered[col].dtype in ['int64', 'float64']:
        numeric_insurance_cols.append(col)

print(f"Numeric insurance columns: {len(numeric_insurance_cols)}")

if len(numeric_insurance_cols) > 0:
    # Create total insurance products
    df_engineered['Total_Insurance_Products'] = df_engineered[numeric_insurance_cols].fillna(0).sum(axis=1)

    print(f"\nTotal Insurance Products distribution:")
    print(df_engineered['Total_Insurance_Products'].describe())

    # Analyze relationship with target
    print(f"\nInsurance Products vs MySavings Target:")
    product_target = df_engineered.groupby('Total_Insurance_Products')['Target'].agg(['count', 'sum', 'mean']).round(4)
    product_target.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    # Only show groups with at least 10 customers
    product_target = product_target[product_target['Customers'] >= 10]
    print(product_target.head(10))

STEP 11 - Insurance Product Features:
Found 35 insurance product columns

First 5 insurance columns sample:
  QTD_APL_VIG_T_S:
    Data type: int64
    Non-null values: 377888
    Sample values: [0, 0, 0]

  QTD_APL_VIG_T_S_AP:
    Data type: int64
    Non-null values: 377888
    Sample values: [0, 0, 0]

  QTD_APL_VIG_T_S_ATCP:
    Data type: int64
    Non-null values: 377888
    Sample values: [0, 0, 0]

  QTD_APL_VIG_T_S_CAP:
    Data type: int64
    Non-null values: 377888
    Sample values: [0, 0, 0]

  QTD_APL_VIG_T_S_PPR:
    Data type: int64
    Non-null values: 377888
    Sample values: [0, 0, 0]

Numeric insurance columns: 35

Total Insurance Products distribution:
count    377888.000000
mean         12.541129
std          10.671509
min           1.000000
25%           6.000000
50%          10.000000
75%          16.000000
max         477.000000
Name: Total_Insurance_Products, dtype: float64

Insurance Products vs MySavings Target:
                          Customers  MySavin

In [44]:
print("STEP 12 - Financial Features (Properly Fixed):")
print("=" * 50)

# Fix CLV column (European format: comma as decimal)
print("Fixing CLV format...")
df_engineered['CLV_numeric'] = df_engineered['CLV'].str.replace('.', '').str.replace(',', '.').astype(float)

print(f"CLV after conversion:")
print(f"  Non-null values: {df_engineered['CLV_numeric'].notna().sum()}")
print(f"  Distribution: {df_engineered['CLV_numeric'].describe()}")

# Create CLV segments
print(f"\nCreating CLV segments...")
df_engineered['CLV_Segment'] = pd.qcut(df_engineered['CLV_numeric'],
                                       q=4, labels=['Low', 'Medium', 'High', 'Premium'],
                                       duplicates='drop')

clv_analysis = df_engineered.groupby('CLV_Segment')['Target'].agg(['count', 'sum', 'mean']).round(4)
clv_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print("CLV segments vs Target:")
print(clv_analysis)

# Use Score columns (these are already clean)
print(f"\nAnalyzing Score Features:")

# CLV Score analysis
print("CLV Score vs Target:")
clv_score_analysis = df_engineered.groupby('VAL_SCORE_CLV')['Target'].agg(['count', 'sum', 'mean']).round(4)
clv_score_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print(clv_score_analysis.sort_values('Conversion_Rate', ascending=False))

# FNOL Score analysis
print(f"\nFNOL Score vs Target:")
fnol_score_analysis = df_engineered.groupby('VAL_SCORE_FNOL')['Target'].agg(['count', 'sum', 'mean']).round(4)
fnol_score_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print(fnol_score_analysis.sort_values('Conversion_Rate', ascending=False))

# Create combined financial health score
print(f"\nCreating combined financial score...")
df_engineered['Financial_Health_Score'] = df_engineered['VAL_SCORE_CLV'] + df_engineered['VAL_SCORE_FNOL']

fin_health_analysis = df_engineered.groupby('Financial_Health_Score')['Target'].agg(['count', 'sum', 'mean']).round(4)
fin_health_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print("Combined Financial Health Score vs Target:")
print(fin_health_analysis.sort_values('Conversion_Rate', ascending=False).head(10))

STEP 12 - Financial Features (Properly Fixed):
Fixing CLV format...
CLV after conversion:
  Non-null values: 350173
  Distribution: count    350173.000000
mean        392.673454
std         691.799093
min       -9901.920000
25%         106.540000
50%         320.460000
75%         632.790000
max        8831.480000
Name: CLV_numeric, dtype: float64

Creating CLV segments...
CLV segments vs Target:
             Customers  MySavings_Buyers  Conversion_Rate
CLV_Segment                                              
Low              87545               172           0.0020
Medium           87542               144           0.0016
High             87545               162           0.0019
Premium          87541               156           0.0018

Analyzing Score Features:
CLV Score vs Target:
               Customers  MySavings_Buyers  Conversion_Rate
VAL_SCORE_CLV                                              
1.0                28717                62           0.0022
2.0                40669

In [45]:
print("STEP 13 - Digital Behavior Features:")
print("=" * 50)

# Find digital behavior columns
digital_cols = [col for col in df_engineered.columns if
                any(x in col.upper() for x in ['LOGIN', 'APP', 'DIGITAL', 'INTERACT', 'DAYS', 'SITE'])]
print(f"Found {len(digital_cols)} digital behavior columns")

# Show some key digital columns
key_digital = ['IND_LOGIN_SITE', 'IND_LOGIN_APP_MYFID', 'QT_DAYS_LAST_LOGIN_SITE', 'QT_INTERACTS_SITE_M1']
available_digital = [col for col in key_digital if col in df_engineered.columns]

print(f"\nKey digital columns available: {available_digital}")

# Check login indicators
login_indicators = [col for col in df_engineered.columns if 'IND_LOGIN' in col]
print(f"\nLogin indicators found: {len(login_indicators)}")

for col in login_indicators[:5]:  # Check first 5
    print(f"\n{col}:")
    print(f"  Values: {df_engineered[col].value_counts()}")

# Create digital engagement score
print(f"\nCreating digital engagement features...")

# Count total number of digital channels used
digital_channels_used = 0
for col in login_indicators:
    if col in df_engineered.columns:
        digital_channels_used += df_engineered[col].fillna(0)

df_engineered['Digital_Channels_Count'] = digital_channels_used

print(f"Digital channels count vs Target:")
digital_analysis = df_engineered.groupby('Digital_Channels_Count')['Target'].agg(['count', 'sum', 'mean']).round(4)
digital_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print(digital_analysis)

# Check recent login activity
recent_login_cols = [col for col in df_engineered.columns if 'QT_DAYS_LAST_LOGIN' in col]
print(f"\nRecent login columns: {len(recent_login_cols)}")

if len(recent_login_cols) > 0:
    print(f"Sample recent login column: {recent_login_cols[0]}")
    sample_col = recent_login_cols[0]
    print(f"  Data type: {df_engineered[sample_col].dtype}")
    print(f"  Distribution: {df_engineered[sample_col].describe()}")

STEP 13 - Digital Behavior Features:
Found 70 digital behavior columns

Key digital columns available: ['IND_LOGIN_SITE', 'IND_LOGIN_APP_MYFID', 'QT_DAYS_LAST_LOGIN_SITE', 'QT_INTERACTS_SITE_M1']

Login indicators found: 8

IND_LOGIN_APP_MYFID:
  Values: IND_LOGIN_APP_MYFID
1    266193
0    111695
Name: count, dtype: int64

IND_LOGIN_APP_DRIVE:
  Values: IND_LOGIN_APP_DRIVE
0    318793
1     59095
Name: count, dtype: int64

IND_LOGIN_APP_MYMLT:
  Values: IND_LOGIN_APP_MYMLT
0    323743
1     54145
Name: count, dtype: int64

IND_LOGIN_APP_VTL:
  Values: IND_LOGIN_APP_VTL
0    341682
1     36206
Name: count, dtype: int64

IND_LOGIN_APP_MYSAV:
  Values: IND_LOGIN_APP_MYSAV
0    331792
1     46096
Name: count, dtype: int64

Creating digital engagement features...
Digital channels count vs Target:
                        Customers  MySavings_Buyers  Conversion_Rate
Digital_Channels_Count                                              
0                               5                 0       

In [46]:
print("STEP 14 - Digital Engagement Segmentation:")
print("=" * 50)


# Create digital engagement segments based on channels used
def create_digital_segments(channels_count):
    if channels_count == 0:
        return 'No_Digital'
    elif channels_count <= 2:
        return 'Low_Digital'
    elif channels_count <= 4:
        return 'Medium_Digital'
    else:
        return 'High_Digital'


df_engineered['Digital_Engagement'] = df_engineered['Digital_Channels_Count'].apply(create_digital_segments)

print("Digital engagement segments vs Target:")
digital_seg_analysis = df_engineered.groupby('Digital_Engagement')['Target'].agg(['count', 'sum', 'mean']).round(4)
digital_seg_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print(digital_seg_analysis.sort_values('Conversion_Rate', ascending=False))

# Create recency segments for main app (MyFid)
print(f"\nCreating app recency segments...")
df_engineered['App_Recency_Segment'] = pd.cut(
    df_engineered['QT_DAYS_LAST_LOGIN_APP_MYFID'].fillna(999),
    bins=[-1, 30, 90, 180, 360, 999],
    labels=['Very_Recent', 'Recent', 'Moderate', 'Old', 'Very_Old']
)

recency_analysis = df_engineered.groupby('App_Recency_Segment')['Target'].agg(['count', 'sum', 'mean']).round(4)
recency_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
print("App recency vs Target:")
print(recency_analysis.sort_values('Conversion_Rate', ascending=False))

STEP 14 - Digital Engagement Segmentation:
Digital engagement segments vs Target:
                    Customers  MySavings_Buyers  Conversion_Rate
Digital_Engagement                                              
High_Digital            22687                87           0.0038
Medium_Digital         124994               286           0.0023
Low_Digital            230202               330           0.0014
No_Digital                  5                 0           0.0000

Creating app recency segments...
App recency vs Target:
                     Customers  MySavings_Buyers  Conversion_Rate
App_Recency_Segment                                              
Very_Recent              86504               212           0.0025
Recent                    5663                11           0.0019
Old                     100729               190           0.0019
Moderate                  7572                14           0.0018
Very_Old                177420               276           0.0016


In [48]:
print("STEP 15 - Feature Engineering Summary:")
print("=" * 60)

# List all new features created
new_features = [
    'Age_Group', 'Total_Insurance_Products', 'CLV_numeric', 'CLV_Segment',
    'Financial_Health_Score', 'Digital_Channels_Count', 'Digital_Engagement',
    'App_Recency_Segment'
]

print("New features created:")
for feature in new_features:
    if feature in df_engineered.columns:
        print(f"  ✓ {feature}")
    else:
        print(f"  ✗ {feature} (missing)")

print(f"\nDataset shape after feature engineering: {df_engineered.shape}")

# Show top converting segments across all new features
print(f"\nTop converting segments (>0.003% conversion rate):")
print("-" * 50)

# Age groups
top_age = df_engineered.groupby('Age_Group')['Target'].mean().sort_values(ascending=False)
print(f"Best age group: {top_age.index[0]} ({top_age.iloc[0]:.4f})")

# Digital engagement
top_digital = df_engineered.groupby('Digital_Engagement')['Target'].mean().sort_values(ascending=False)
print(f"Best digital engagement: {top_digital.index[0]} ({top_digital.iloc[0]:.4f})")

# Financial health score
top_financial = df_engineered.groupby('Financial_Health_Score')['Target'].mean().sort_values(ascending=False)
print(f"Best financial score: {top_financial.index[0]} ({top_financial.iloc[0]:.4f})")

STEP 15 - Feature Engineering Summary:
New features created:
  ✓ Age_Group
  ✓ Total_Insurance_Products
  ✓ CLV_numeric
  ✓ CLV_Segment
  ✓ Financial_Health_Score
  ✓ Digital_Channels_Count
  ✓ Digital_Engagement
  ✓ App_Recency_Segment

Dataset shape after feature engineering: (377888, 565)

Top converting segments (>0.003% conversion rate):
--------------------------------------------------
Best age group: 18-24 (0.0037)
Best digital engagement: High_Digital (0.0038)
Best financial score: 11.0 (0.0023)


In [49]:
print("STEP 16 - Product Combination Analysis:")
print("=" * 50)

# Analyze which insurance products are most common together
insurance_product_cols = [col for col in df_engineered.columns if 'QTD_APL_VIG_T_' in col and col != 'QTD_APL_VIG_T_S']

print(f"Found {len(insurance_product_cols)} specific insurance products")

# Show first few to understand the product types
for col in insurance_product_cols[:8]:
    non_zero = (df_engineered[col] > 0).sum()
    print(f"  {col}: {non_zero:,} customers ({non_zero / len(df_engineered) * 100:.1f}%)")

# Create product diversity score
print(f"\nCreating product diversity features...")

# Count how many different product types each customer has
df_engineered['Product_Diversity'] = (df_engineered[insurance_product_cols] > 0).sum(axis=1)

# Create product concentration (is portfolio concentrated in few products?)
total_products = df_engineered[insurance_product_cols].sum(axis=1)
max_single_product = df_engineered[insurance_product_cols].max(axis=1)
df_engineered['Product_Concentration'] = max_single_product / (total_products + 0.01)  # Avoid division by zero

print("Product diversity vs Target:")
diversity_analysis = df_engineered.groupby('Product_Diversity')['Target'].agg(['count', 'sum', 'mean']).round(4)
diversity_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
# Show only groups with sufficient data
diversity_analysis = diversity_analysis[diversity_analysis['Customers'] >= 100]
print(diversity_analysis.head(10))

STEP 16 - Product Combination Analysis:
Found 19 specific insurance products
  QTD_APL_VIG_T_S_AP: 21,474 customers (5.7%)
  QTD_APL_VIG_T_S_ATCP: 10,714 customers (2.8%)
  QTD_APL_VIG_T_S_CAP: 62,926 customers (16.7%)
  QTD_APL_VIG_T_S_PPR: 68,284 customers (18.1%)
  QTD_APL_VIG_T_S_SD: 68,123 customers (18.0%)
  QTD_APL_VIG_T_S_VDR: 26,226 customers (6.9%)
  QTD_APL_VIG_T_AP: 23,875 customers (6.3%)
  QTD_APL_VIG_T_ATCO: 10,038 customers (2.7%)

Creating product diversity features...
Product diversity vs Target:
                   Customers  MySavings_Buyers  Conversion_Rate
Product_Diversity                                              
0                        933                 0           0.0000
1                     135803               141           0.0010
2                      91903               133           0.0014
3                      64512               148           0.0023
4                      35656                98           0.0027
5                      21485    

In [50]:
print("STEP 17 - Time-Based Behavior Patterns:")
print("=" * 50)

# Find recency columns for different behaviors
recency_cols = [col for col in df_engineered.columns if 'QTD_MESES_ULT' in col]
print(f"Found {len(recency_cols)} recency columns")

# Show key recency patterns
for col in recency_cols[:5]:
    print(f"\n{col}:")
    print(f"  Non-null: {df_engineered[col].notna().sum()}")
    if df_engineered[col].notna().sum() > 0:
        print(f"  Distribution: {df_engineered[col].describe()}")

# Create customer lifecycle stage based on tenure and recent activity
print(f"\nCreating customer lifecycle features...")

# Recent activity score (lower = more recent activity)
if 'QTD_MESES_ULT_COMPRA_APL' in df_engineered.columns:
    df_engineered['Recent_Purchase_Activity'] = df_engineered['QTD_MESES_ULT_COMPRA_APL'].fillna(999)


    # Create lifecycle segments
    def create_lifecycle_stage(tenure, recent_activity):
        if tenure < 2:
            return 'New_Customer'
        elif recent_activity <= 6:
            return 'Active_Customer'
        elif recent_activity <= 24:
            return 'Moderate_Customer'
        else:
            return 'Dormant_Customer'


    df_engineered['Lifecycle_Stage'] = df_engineered.apply(
        lambda x: create_lifecycle_stage(x['QTD_ANOS_ANTIG_CLIENTE'], x['Recent_Purchase_Activity']),
        axis=1
    )

    lifecycle_analysis = df_engineered.groupby('Lifecycle_Stage')['Target'].agg(['count', 'sum', 'mean']).round(4)
    lifecycle_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    print("Customer lifecycle vs Target:")
    print(lifecycle_analysis.sort_values('Conversion_Rate', ascending=False))

STEP 17 - Time-Based Behavior Patterns:
Found 24 recency columns

QTD_MESES_ULT_CONT:
  Non-null: 377888
  Distribution: count    377888.000000
mean         13.268863
std          15.465711
min           0.000000
25%           1.000000
50%           8.000000
75%          21.000000
max         200.000000
Name: QTD_MESES_ULT_CONT, dtype: float64

QTD_MESES_ULT_COMPRA_APL:
  Non-null: 377888
  Distribution: count    377888.000000
mean         46.720150
std          49.448658
min          -1.000000
25%          12.000000
50%          31.000000
75%          63.000000
max        1225.000000
Name: QTD_MESES_ULT_COMPRA_APL, dtype: float64

QTD_MESES_ULT_COMPRA_AP:
  Non-null: 97575
  Distribution: count    97575.000000
mean       170.636618
std        130.199750
min          0.000000
25%         49.000000
50%        153.000000
75%        274.000000
max       1027.000000
Name: QTD_MESES_ULT_COMPRA_AP, dtype: float64

QTD_MESES_ULT_COMPRA_ATCO:
  Non-null: 22188
  Distribution: count    22188.00

In [51]:
print("STEP 18 - Geographic Intelligence:")
print("=" * 50)

# Find geographic columns
geo_cols = [col for col in df_engineered.columns if
            any(x in col.upper() for x in ['DISTRITO', 'CONCELHO', 'NUT', 'RURAL', 'LITORAL'])]
print(f"Found {len(geo_cols)} geographic columns")

# Check what geographic data we have
key_geo_cols = ['DSC_DISTRITO', 'DSC_CONCELHO', 'DSC_RURAL_URBANO', 'DSC_INTERIOR_LITORAL']
available_geo = [col for col in key_geo_cols if col in df_engineered.columns]

print(f"Available geographic columns: {available_geo}")

for col in available_geo:
    print(f"\n{col}:")
    top_values = df_engineered[col].value_counts().head(5)
    print(f"  Top values: {top_values.to_dict()}")

# Create geographic risk segments
if 'DSC_RURAL_URBANO' in df_engineered.columns:
    print(f"\nUrban vs Rural analysis:")
    urban_rural_analysis = df_engineered.groupby('DSC_RURAL_URBANO')['Target'].agg(['count', 'sum', 'mean']).round(4)
    urban_rural_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    print(urban_rural_analysis)

if 'DSC_INTERIOR_LITORAL' in df_engineered.columns:
    print(f"\nInterior vs Coastal analysis:")
    interior_analysis = df_engineered.groupby('DSC_INTERIOR_LITORAL')['Target'].agg(['count', 'sum', 'mean']).round(4)
    interior_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    print(interior_analysis)

# Create district-level conversion rates (for district encoding)
if 'DSC_DISTRITO' in df_engineered.columns:
    print(f"\nTop converting districts:")
    district_analysis = df_engineered.groupby('DSC_DISTRITO')['Target'].agg(['count', 'sum', 'mean']).round(4)
    district_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    district_analysis = district_analysis[district_analysis['Customers'] >= 1000]  # Only districts with sufficient data
    district_analysis = district_analysis.sort_values('Conversion_Rate', ascending=False)
    print(district_analysis.head(10))

    # Create district performance encoding
    district_performance = district_analysis['Conversion_Rate'].to_dict()
    df_engineered['District_Performance'] = df_engineered['DSC_DISTRITO'].map(district_performance).fillna(
        df_engineered['Target'].mean())

STEP 18 - Geographic Intelligence:
Found 10 geographic columns
Available geographic columns: ['DSC_DISTRITO', 'DSC_CONCELHO', 'DSC_RURAL_URBANO', 'DSC_INTERIOR_LITORAL']

DSC_DISTRITO:
  Top values: {'LISBOA': 103950, 'PORTO': 58709, 'SETÚBAL': 34722, 'BRAGA': 26380, 'AVEIRO': 25415}

DSC_CONCELHO:
  Top values: {'LISBOA': 28077, 'SINTRA': 16027, 'CASCAIS': 10288, 'OEIRAS': 10201, 'VILA NOVA DE GAIA': 9470}

DSC_RURAL_URBANO:
  Top values: {'ÁREA PREDOMINANTEMENTE URBANA': 191911, 'ÁREA MEDIAMENTE URBANA': 119304, 'ÁREA PREDOMINANTEMENTE RURAL': 65453}

DSC_INTERIOR_LITORAL:
  Top values: {'INTERIOR': 199629, 'LITORAL': 177039}

Urban vs Rural analysis:
                               Customers  MySavings_Buyers  Conversion_Rate
DSC_RURAL_URBANO                                                           
ÁREA MEDIAMENTE URBANA            119304               260           0.0022
ÁREA PREDOMINANTEMENTE RURAL       65453               126           0.0019
ÁREA PREDOMINANTEMENTE URBANA     

In [52]:
print("STEP 19 - Advanced Interaction Features:")
print("=" * 50)

# Create powerful interaction features based on our best predictors
print("Creating interaction features...")

# Age × Digital engagement interaction
df_engineered['Age_Digital_Score'] = df_engineered['QTD_IDADE'] * df_engineered['Digital_Channels_Count']

# Financial health × Product diversity interaction
df_engineered['Financial_Product_Score'] = df_engineered['Financial_Health_Score'] * df_engineered['Product_Diversity']

# Tenure × Recent activity interaction (engagement decay)
df_engineered['Tenure_Activity_Ratio'] = df_engineered['QTD_ANOS_ANTIG_CLIENTE'] / (
            df_engineered['Recent_Purchase_Activity'] + 1)

# High-value digital customer flag
df_engineered['High_Value_Digital'] = (
        (df_engineered['Digital_Engagement'] == 'High_Digital') &
        (df_engineered['CLV_Segment'] == 'Premium')
).astype(int)

# Young active customer flag
df_engineered['Young_Active_Customer'] = (
        (df_engineered['Age_Group'] == '18-24') &
        (df_engineered['Lifecycle_Stage'] == 'Active_Customer')
).astype(int)

print("Analyzing interaction features:")

# Test high-value digital customers
if df_engineered['High_Value_Digital'].sum() > 0:
    hvd_analysis = df_engineered.groupby('High_Value_Digital')['Target'].agg(['count', 'sum', 'mean']).round(4)
    hvd_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    print("High Value Digital customers:")
    print(hvd_analysis)

# Test young active customers
if df_engineered['Young_Active_Customer'].sum() > 0:
    yac_analysis = df_engineered.groupby('Young_Active_Customer')['Target'].agg(['count', 'sum', 'mean']).round(4)
    yac_analysis.columns = ['Customers', 'MySavings_Buyers', 'Conversion_Rate']
    print("\nYoung Active customers:")
    print(yac_analysis)

# Show correlation of new interaction features with target
interaction_features = ['Age_Digital_Score', 'Financial_Product_Score', 'Tenure_Activity_Ratio', 'High_Value_Digital',
                        'Young_Active_Customer']
print(f"\nCorrelation with Target:")
for feature in interaction_features:
    if feature in df_engineered.columns:
        corr = df_engineered[feature].corr(df_engineered['Target'])
        print(f"  {feature}: {corr:.4f}")

STEP 19 - Advanced Interaction Features:
Creating interaction features...
Analyzing interaction features:
High Value Digital customers:
                    Customers  MySavings_Buyers  Conversion_Rate
High_Value_Digital                                              
0                      372017               680           0.0018
1                        5871                23           0.0039

Young Active customers:
                       Customers  MySavings_Buyers  Conversion_Rate
Young_Active_Customer                                              
0                         377020               699           0.0019
1                            868                 4           0.0046

Correlation with Target:
  Age_Digital_Score: 0.0104
  Financial_Product_Score: 0.0158
  Tenure_Activity_Ratio: nan
  High_Value_Digital: 0.0060
  Young_Active_Customer: 0.0031


In [53]:
print("STEP 20 - Advanced Feature Engineering Summary:")
print("=" * 60)

# List all advanced features created
advanced_features = [
    'Product_Diversity', 'Product_Concentration', 'Lifecycle_Stage',
    'District_Performance', 'Age_Digital_Score', 'Financial_Product_Score',
    'High_Value_Digital', 'Young_Active_Customer'
]

print("Advanced features created:")
for feature in advanced_features:
    if feature in df_engineered.columns:
        print(f"  ✓ {feature}")
    else:
        print(f"  ✗ {feature} (missing)")

# Final dataset summary
print(f"\nFinal dataset shape: {df_engineered.shape}")
print(f"Total features available: {df_engineered.shape[1] - 1}")  # Minus target

# Top predictive features summary
print(f"\nTop predictive patterns discovered:")
print(f"  🏆 Young Active Customers: 0.46% conversion (2.4x baseline)")
print(f"  🏆 High Value Digital: 0.39% conversion (2.2x baseline)")
print(f"  🏆 Product Diversity 8+: 0.46% conversion (4.6x baseline)")
print(f"  🏆 High Digital Engagement: 0.38% conversion (2.0x baseline)")
print(f"  🏆 Age 18-24: 0.37% conversion (1.9x baseline)")

# Check for any missing values in key engineered features
print(f"\nMissing values in engineered features:")
key_engineered = ['Age_Group', 'Digital_Engagement', 'Lifecycle_Stage', 'CLV_Segment', 'Product_Diversity']
for feature in key_engineered:
    if feature in df_engineered.columns:
        missing_pct = (df_engineered[feature].isnull().sum() / len(df_engineered)) * 100
        print(f"  {feature}: {missing_pct:.1f}%")

STEP 20 - Advanced Feature Engineering Summary:
Advanced features created:
  ✓ Product_Diversity
  ✓ Product_Concentration
  ✓ Lifecycle_Stage
  ✓ District_Performance
  ✓ Age_Digital_Score
  ✓ Financial_Product_Score
  ✓ High_Value_Digital
  ✓ Young_Active_Customer

Final dataset shape: (377888, 575)
Total features available: 574

Top predictive patterns discovered:
  🏆 Young Active Customers: 0.46% conversion (2.4x baseline)
  🏆 High Value Digital: 0.39% conversion (2.2x baseline)
  🏆 Product Diversity 8+: 0.46% conversion (4.6x baseline)
  🏆 High Digital Engagement: 0.38% conversion (2.0x baseline)
  🏆 Age 18-24: 0.37% conversion (1.9x baseline)

Missing values in engineered features:
  Age_Group: 0.0%
  Digital_Engagement: 0.0%
  Lifecycle_Stage: 0.0%
  CLV_Segment: 7.3%
  Product_Diversity: 0.0%


In [54]:
print("STEP 21 - Ready for Modeling Assessment:")
print("=" * 60)

# Categorize all our features for modeling prep
feature_types = {
    'Target': ['Target'],
    'ID_Columns': ['ID_CLIENTE_ANON', 'ANO_MES'],
    'Original_Numeric': [],
    'Original_Categorical': [],
    'Engineered_Numeric': ['Total_Insurance_Products', 'CLV_numeric', 'Financial_Health_Score',
                           'Digital_Channels_Count', 'Product_Diversity', 'Age_Digital_Score',
                           'Financial_Product_Score'],
    'Engineered_Categorical': ['Age_Group', 'CLV_Segment', 'Digital_Engagement',
                               'App_Recency_Segment', 'Lifecycle_Stage'],
    'Engineered_Binary': ['High_Value_Digital', 'Young_Active_Customer']
}

# Count features in each category
for category, features in feature_types.items():
    available = [f for f in features if f in df_engineered.columns]
    print(f"{category}: {len(available)} features")

# Check if we're ready for modeling
modeling_readiness = {
    'Target_Variable': 'Target' in df_engineered.columns,
    'Numeric_Features': len([f for f in feature_types['Engineered_Numeric'] if f in df_engineered.columns]) > 0,
    'Categorical_Features': len([f for f in feature_types['Engineered_Categorical'] if f in df_engineered.columns]) > 0,
    'No_Major_Missing': all(df_engineered[f].isnull().sum() < len(df_engineered) * 0.5 for f in
                            ['Target', 'Age_Group', 'Digital_Engagement'])
}

print(f"\nModeling readiness check:")
for check, status in modeling_readiness.items():
    status_icon = "✅" if status else "❌"
    print(f"  {status_icon} {check}")

all_ready = all(modeling_readiness.values())
print(f"\n{'🚀 READY FOR MODELING!' if all_ready else '⚠️ Need to fix issues before modeling'}")

STEP 21 - Ready for Modeling Assessment:
Target: 1 features
ID_Columns: 1 features
Original_Numeric: 0 features
Original_Categorical: 0 features
Engineered_Numeric: 7 features
Engineered_Categorical: 5 features
Engineered_Binary: 2 features

Modeling readiness check:
  ✅ Target_Variable
  ✅ Numeric_Features
  ✅ Categorical_Features
  ✅ No_Major_Missing

🚀 READY FOR MODELING!
